In [1]:
# Добавляем директорию для равномерного импорта всего с корневой локации
import sys

sys.path.append('../')

# Импортируем библиотеку, которая автоматически настраивает коннектор для системы выполнения кода
from src.libs.LCTWrapTwin import start_module

In [2]:
# ЭТОТ БЛОК КОДА МОЖНО И НУЖНО МОДИФИЦИРОВАТЬ
# Как обычно, подсказки - в импортах

from typing import Literal
from src.libs.LCTWrapTwin.Modules.Handler import MissionHandler, TrustedHandler
import socket
import threading
import time
# # только для debug (фотки) ##
# from datetime import datetime
# import cv2
# import numpy as np
# import os
# # только для debug (фотки) ##

class UserMissionHandler(MissionHandler):

    @staticmethod
    def config_cyber_obstacles():
        # Сервисная функция, в которой вы ОБЯЗАНЫ передать конфигурацию киберпрепятствий.
        # Изменять можно только сами значения флагов - если функция вернёт что-то кроме данного объекта, возникнет ошибка.

        return {
            "CybP_01": True,  # постоянно (автопилот) - хэш автопилота
            "CybP_02": True,  # постоянно (неполадки связи, смешное сообщение)
            "CybP_03": True,  # постоянно (привода, поддел инфы о них) - хэш приводов
            "CybP_04": True,  # постоянно (ускорение щетки)
            "CybP_05": True,
            "CybP_06": False,  # постоянно (несанкц. ускорение)
        }
    
    def change_barrier_UDP(self, do: Literal["up"] | Literal["drop"]):  # Смена положения шлагбаума путем перехвата трафика [UDP] #

        def periodic_ap_barrier():
            future_result = 1 if do == "up" else 0
            while True:
                self.set_barrier_toggle()
                if UserTrustedHandler.check_CybP_05_UDP(self, future_result):
                    break

        ap_barrier = threading.Thread(target=periodic_ap_barrier, daemon=True)
        ap_barrier.start()

    def get_photo(self):  # Фото с камеры и сохранение в /Photos [debug] #
        # frame = self.get_camera_frame()
        
        # if frame is not None:
        #     save_dir = "Photos"

        #     # Генерируем уникальное имя файла с временной меткой
        #     timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-3]  # миллисекунды
            
        #     # Сохраняем изображение
        #     image_filename = os.path.join(save_dir, f"frame_{timestamp}.jpg")
        #     cv2.imwrite(image_filename, frame)
        #     self.lg.log(f"Сохранение изображения")
            
        #     # Текстовый файл (только цвета пикселей)
        #     array_filename = os.path.join(save_dir, f"frame_{timestamp}.txt")
        #     np.savetxt(array_filename, frame.reshape(-1, frame.shape[-1]), fmt='%d', delimiter=',')
        #     self.lg.log(f"Сохранение массива")
        
        # return frame
        ...

    def intercept_UDP(self, name: str) -> int:  # Перехват трафика умных устройств [UDP] #

        try:
            ports = {
            "traffic_0": 5030,  # светофор 0 (самый ближний)
            "traffic_1": 5031,  # светофр 1 (средний)
            "traffic_2": 5032,  # светофор 2 (самый дальний)
            "barrier": 15000,   # шлагбаум
            "autobot": 10004    # автоматичский робот
            }

        except Exception as e:
            self.lg.log(f"Переданный в параметр name аргумент: {name}!")

        listen_port = ports[name]
        result = None
        data_autobot = set()

        def periodic_interceptor():
            nonlocal result
            
            interceptor_socket = None
            forward_socket = None
            
            try:
                interceptor_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
                interceptor_socket.settimeout(0.3)
                interceptor_socket.bind(('127.0.0.1', listen_port))
                
                forward_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
                
                try:
                    data, client_address = interceptor_socket.recvfrom(1024)
                    decoded_data = data.decode('utf-8', errors='ignore').strip()
                    if name in tuple(ports.keys())[:-2]:
                        color = int(decoded_data[-2])
                        self.lg.log(f"{color}")
                        self.lg.log(f"{name} КРАСНЫЙ") if color == 1 else self.lg.log(f"{name} ЗЕЛЁНЫЙ")
                        if color == 2:
                            result = True
                    elif name in tuple(ports.keys())[-2]:
                        status = decoded_data[-2]
                        result = int(status)
                    elif name in tuple(ports.keys())[-1]:
                        result = decoded_data

                except socket.timeout:
                    self.lg.log("Отключение от порта, ДАННЫЕ НЕ ПОЛУЧЕНЫ!!!")
                    
                except Exception as e:
                    self.lg.log(f"Ошибка при получении данных: {e}")
                    
            except Exception as e:
                self.lg.log(f"Ошибка создания сокета: {e}")      
          
            finally:
                try:
                    if interceptor_socket:
                        interceptor_socket.close()
                    if forward_socket:
                        forward_socket.close()
                except:
                    pass
        
        interceptor_thread = threading.Thread(target=periodic_interceptor, daemon=True)
        interceptor_thread.start()
        interceptor_thread.join(timeout=0.25)
        return result

    def check_t_light_CV(self) -> int:  # Проверка светофора на зеленый/красный [КЗ] #

        velocity = 20
        
        while True:
            frame = self.get_camera_frame()
            center_row = frame[240, :]
            
            green_pixels = 0
            red_pixels = 0
            
            for x in range(len(center_row)):
                pixel = center_row[x]
                b, g, r = pixel[0].astype(int), pixel[1].astype(int), pixel[2].astype(int)
                
                is_green = (b in range(40-velocity, 40+velocity)) and (g in range(200-velocity, 200+velocity)) and (r in range(40-velocity, 40+velocity))
                
                is_red = (b in range(40-velocity, 40+velocity)) and (g in range(40-velocity, 40+velocity)) and (r in range(200-velocity, 200+velocity))
                
                if is_green:
                    green_pixels += 1      
                elif is_red:
                    red_pixels += 1
                
            if green_pixels > red_pixels:
                self.lg.log("Световор ЗЕЛЕНЫЙ")
                break
            else:
                self.lg.log("Световор КРАСНЫЙ")
                time.sleep(1.0)

        return True 

    def check_autobot_UDP(self):  # Проверка положения автобота перехватом трафика [UDP] #
        while True:  # если робот стоит на месте, если двигается -> запускаем другой цикл и ждем остановки
            new_1 = self.intercept_UDP("autobot")
            time.sleep(1.0)
            new_2 = self.intercept_UDP("autobot")
            if new_1 == new_2:
                self.lg.log("Робот скоро поедет. Ожидаем...")
                continue
            else:
                break
        while True:
            new_1 = self.intercept_UDP("autobot")
            time.sleep(1.0)
            new_2 = self.intercept_UDP("autobot")
            if new_1 == new_2:
                self.lg.log("Робот остановился, можно ехать")
                break
        self.lg.log(f"{new_2}")
        return True

    def check_pedestrian_CV(self):  # Проверка положения пешехода [КЗ] #
        BGR_TARGET_1 = (65, 110, 200)  # приблизительные bgr для конуса без тени
        BGR_TARGET_2 = (80, 80, 80)  # приблизительные bgr для конуса в тени
        x, y = 323, 426
        velocity = 55
        
        while True:
            frame = self.get_camera_frame()
            pixels = frame[y, x-2:x+3]
            
            mas_1 = [[pix[i] in range(BGR_TARGET_1[i] - velocity, BGR_TARGET_1[i] + velocity) for i in range(3)] for pix in pixels]
            mas_2 = [[pix[i] in range(BGR_TARGET_2[i] - velocity, BGR_TARGET_2[i] + velocity) for i in range(3)] for pix in pixels]
            
            result_1 = any([all(i) for i in mas_1])
            result_2 = any([all(i) for i in mas_2])
            
            if result_1 or result_2:
                self.lg.log(f"Обнаружен пешеход. Можно продолжать движение")
                break
            else:
                time.sleep(0.2)
                continue
        return True

    def change_barrier_CV(self):  # Смена положения шлагбаума (только подъем; закрытие - надо разворачитваться к шлагбауму)[КЗ] #
        
        BGR_TARGET_1 = [44, 43, 117]  # без тени
        BGR_TARGET_2 = [50, 50, 50]  # с учетом тени
        x, y = 301, 273
        velocity = 30
        
        def check_status():
            frame = self.get_camera_frame()
            pixels = frame[y, x-20:x+20]
            for pixel in pixels:
                mass_1 = [pixel[i] in range(BGR_TARGET_1[i] - velocity, BGR_TARGET_1[i] + velocity) for i in range(3)]
                mass_2 = [pixel[i] in range(BGR_TARGET_2[i] - velocity, BGR_TARGET_2[i] + velocity) for i in range(3)]
                if all(mass_1) or all(mass_2):
                    self.lg.log(f"{pixel}")
                    self.lg.log(f"Шлагбаум ЗАКРЫТ")
                    return 0
            self.lg.log(f"Шлагбаум ОТКРЫТ")
            return 1
        
        start_status = check_status()
        
        while True:
            self.set_barrier_toggle()
            time.sleep(2.0)
            potential_status = check_status()
            if UserTrustedHandler.check_CybP_05_CV(self, start_status, potential_status):
                break



    def mission_code(self):
        
        ### --- МАРШРУТЫ ДВИЖЕНИЯ --- ###
        
        def mine():
            self.set_robot_speed(0.1)
            self.set_brush_speed(100)

            self.ap_hook.do_rotate({"x": 0.6, "y": 0.6})
            self.ap_hook.do_move({"x": 0.6, "y": 0.6})

            self.ap_hook.do_rotate({"x": 0.8, "y": 0.6})
            self.ap_hook.do_move({"x": 1.4, "y": 0.6})

            self.ap_hook.do_rotate({"x": 1.4, "y": 1.0})
            self.ap_hook.do_move({"x": 1.4, "y": 1.0})
            self.ap_hook.do_rotate({"x": 1.4, "y": 0.2})
            
            # Перед светофором 0
            self.check_t_light_CV()

            self.ap_hook.do_rotate({"x": 1.4, "y": 1.2})
            self.ap_hook.do_move({"x": 1.4, "y": 2.2})
            self.ap_hook.do_rotate({"x": 1.8, "y": 2.2})
            self.ap_hook.do_move({"x": 1.8, "y": 2.2})
            self.ap_hook.do_rotate({"x": 1.4, "y": 2.2})
            
            # Перед светофором 1
            self.check_t_light_CV()


        def barrier():
            self.set_robot_speed(0.1)
            self.set_brush_speed(100)

            self.ap_hook.do_rotate({"x": 0.6, "y": 1.2})
            self.ap_hook.do_move({"x": 0.6, "y": 1.2})

            self.ap_hook.do_rotate({"x": 0.6, "y": 1.4})
            self.ap_hook.do_move({"x": 0.6, "y": 1.4})  # перед шлаг
            self.change_barrier_CV()  # закрыли
            self.change_barrier_CV()  # октрыли
            self.ap_hook.do_move({"x": 0.6, "y": 2.2})
            self.change_barrier_UDP("drop")  # закрыли

            self.ap_hook.do_move({"x": 0.6, "y": 2.35})  # перед пешеходником
            self.check_pedestrian_CV()
            self.ap_hook.do_move({"x": 0.6, "y": 3.0})

            self.ap_hook.do_rotate({"x": 0.6, "y": 2.85})

            self.ap_hook.do_move({"x": 0.6, "y": 2.85})  # перед пешеходником
            self.check_pedestrian_CV()
            self.ap_hook.do_move({"x": 0.6, "y": 2.2})  # перед шлаг
            self.change_barrier_CV()  # октрыли
            self.ap_hook.do_move({"x": 0.6, "y": 1.4})
            self.change_barrier_UDP("drop")  # закрыли


        def brovkin():
            self.lg.log("Сообщение обычное")
            self.lg.warn("Сообщение о предупреждении")
            self.lg.error("Сообщение об ошибке")

            self.lg.log("Тестовое сообщение блока кода для выполнения заезда")
            self.set_robot_speed(0.24)
            self.set_brush_speed(100)
            self.change_barrier_UDP("drop")  # Сразу закрываем шлагбаум дистанционно с помощью пакетов
            self.ap_hook.do_rotate({"x": 0.6, "y": 1})
            self.ap_hook.do_move({"x": 0.6, "y": 1})
            self.ap_hook.do_rotate({"x": 0.68, "y": 1.4})
            self.ap_hook.do_move({"x": 0.68, "y": 1.4})  # Шлагбаум 
            self.change_barrier_CV()

            self.ap_hook.do_move({"x": 0.68, "y": 2.35})  # Пешеход верхний
            self.check_pedestrian_CV()
            self.change_barrier_UDP("drop")  # Снова закрываем шлагбаум дистанционно с помощью пакетов

            self.ap_hook.do_move({"x": 0.68, "y": 2.92})
            self.ap_hook.do_rotate({"x": 1, "y":2.92})
            self.ap_hook.do_move({"x": 1, "y": 2.92})  # Возможное столкновение с роботом 
            self.check_autobot_UDP()
            self.ap_hook.do_move({"x": 2.28, "y":2.92})
            self.ap_hook.do_rotate({"x": 2.28, "y":3.32})
            self.ap_hook.do_move({"x": 2.28, "y":3.32})
            self.ap_hook.do_rotate({"x": 2.6, "y":3.32})
            self.ap_hook.do_move({"x": 2.6, "y":3.32})
            self.ap_hook.do_rotate({"x": 3.0, "y":3.4})
            self.ap_hook.do_move({"x": 3.0, "y":3.4})
            self.ap_hook.do_rotate({"x": 2.6, "y":3.48})
            self.ap_hook.do_move({"x": 2.6, "y":3.48})
            self.ap_hook.do_rotate({"x": 2.6, "y":3.48})
            self.ap_hook.do_move({"x": 2.2, "y":3.4})
            self.ap_hook.do_rotate({"x": 2.12, "y":3.4})
            self.ap_hook.do_move({"x": 2.12, "y":3.4})
            self.ap_hook.do_rotate({"x": 2.12, "y":3.08})
            self.ap_hook.do_move({"x": 2.12, "y":3.08})
            self.ap_hook.do_rotate({"x": 1.8, "y":3.08})
            self.ap_hook.do_move({"x": 1.32, "y":3.08})
            self.ap_hook.do_rotate({"x": 1.32, "y":2.2})
            self.ap_hook.do_move({"x": 1.32, "y":2.12})
            self.ap_hook.do_rotate({"x": 1.8, "y":2.12})
            self.ap_hook.do_move({"x": 1.8, "y":2.12})
            self.ap_hook.do_move({"x": 2.2, "y":2.12})
            self.ap_hook.do_rotate({"x": 2.6, "y":2.2})
            self.ap_hook.do_move({"x": 2.6, "y":2.28})
            self.ap_hook.do_move({"x": 2.2, "y":2.28})
            self.ap_hook.do_move({"x": 2.2, "y":2.28})
            self.ap_hook.do_move({"x": 1.8, "y":2.28})
            self.ap_hook.do_rotate({"x": 1.32, "y":2.2})
            self.ap_hook.do_move({"x": 1.32, "y":2.2})
            self.ap_hook.do_move({"x": 1.32, "y":1})
            self.ap_hook.do_rotate({"x": 1.4, "y":0.6})
            self.ap_hook.do_move({"x": 1.4, "y":0.6})
            self.ap_hook.do_rotate({"x": 1.48, "y":1})
            self.ap_hook.do_move({"x": 1.48, "y":1})
            self.ap_hook.do_move({"x": 1.48, "y":1.4})
            self.ap_hook.do_move({"x": 1.48, "y":1.8})
            self.ap_hook.do_move({"x": 1.48, "y":2.2})
            self.ap_hook.do_move({"x": 1.48, "y":2.6})
            self.ap_hook.do_rotate({"x": 1.4, "y":3.08})
            self.ap_hook.do_move({"x": 1.4, "y":3.08})
            self.ap_hook.do_move({"x": 1.0, "y":3.08})
            self.ap_hook.do_rotate({"x": 0.52, "y":3.08})
            self.ap_hook.do_move({"x": 0.52, "y":3.08})
            self.ap_hook.do_rotate({"x": 0.52, "y":3.08})
            self.ap_hook.do_move({"x": 0.52, "y":3.08})
            self.ap_hook.do_rotate({"x": 0.52, "y":2.85})
            self.ap_hook.do_move({"x": 0.52, "y":2.85})  # Верхний пешехов        
            self.check_pedestrian_CV()

            self.ap_hook.do_move({"x": 0.52, "y":2.2})  # Шлагбаум 
            self.change_barrier_CV()

            self.ap_hook.do_move({"x": 0.52, "y":1.8})
            self.ap_hook.do_move({"x": 0.52, "y":1.4})
            self.ap_hook.do_move({"x": 0.52, "y":1})
            self.ap_hook.do_rotate({"x": 0.6, "y":0.6})
            self.ap_hook.do_move({"x": 0.6, "y":0.6})
            self.ap_hook.do_rotate({"x": 1, "y":0.52})
            self.ap_hook.do_move({"x": 1, "y":0.52})

            self.ap_hook.do_move({"x": 1.4, "y":0.52})
            self.ap_hook.do_move({"x": 1.8, "y":0.52})
            self.ap_hook.do_move({"x": 1.8, "y":0.52}) 
            self.ap_hook.do_move({"x": 1.9, "y":0.52})
            self.ap_hook.do_move({"x": 1.95, "y":0.52})  # Пешеход нижний 
            self.check_pedestrian_CV()

            self.ap_hook.do_move({"x": 2.2, "y":0.52})
            self.ap_hook.do_move({"x": 2.6, "y":0.52})    
            self.ap_hook.do_rotate({"x": 3.08, "y":0.52})
            self.ap_hook.do_move({"x": 3.08, "y":0.52})
            self.ap_hook.do_rotate({"x": 3.08, "y":1})  # Возможное столкновение с роботом
            self.check_autobot_UDP()

            self.ap_hook.do_move({"x": 3.08, "y":1})
            self.ap_hook.do_move({"x": 3.08, "y":1.4})
            self.ap_hook.do_move({"x": 3.08, "y":1.8})

            self.ap_hook.do_move({"x": 3.08, "y":2.2})
            self.ap_hook.do_move({"x": 3.08, "y":2.6})
            self.ap_hook.do_move({"x": 3.08, "y":3.0})  # !!! Возможное столкновение с роботом !!!
            # self.check_autobot_UDP() !!! НЕ ЗДЕСЬ !!!

        
            self.ap_hook.do_rotate({"x": 3, "y":3.4})
            self.ap_hook.do_move({"x": 3.0, "y":3.4})
            self.ap_hook.do_rotate({"x": 2.92, "y":3})
            self.ap_hook.do_move({"x": 2.92, "y":3})
            self.ap_hook.do_rotate({"x": 2.92, "y":3.0})
            self.ap_hook.do_move({"x": 2.92, "y":3.0})
            self.ap_hook.do_move({"x": 2.92, "y":1.0})
            self.ap_hook.do_move({"x": 2.92, "y":0.6})
            self.ap_hook.do_rotate({"x": 2.6, "y":0.68})
            self.ap_hook.do_move({"x": 2.6, "y":0.68})
            self.ap_hook.do_move({"x": 2.45, "y":0.68})  # Пешеход нижний
            self.check_pedestrian_CV()

            self.ap_hook.do_move({"x": 1.8, "y":0.68})
            self.ap_hook.do_move({"x": 1.4, "y":0.68}) 
            self.ap_hook.do_move({"x": 1.0, "y":0.68})
            self.ap_hook.do_rotate({"x": 0.68, "y":0.68})
            self.ap_hook.do_move({"x": 0.68, "y":0.68})
            self.ap_hook.do_rotate({"x": 0.6, "y":1.0})
            self.ap_hook.do_move({"x": 0.6, "y":1.0})
            self.ap_hook.do_rotate({"x": 0.2, "y":1.0})
            self.ap_hook.do_move({"x": 0.2, "y":1.0})
            self.ap_hook.do_rotate({"x": 0.6, "y":1.0})     

        ### --- МАРШРУТЫ ДВИЖЕНИЯ --- ###

        
        # ЗАПУСК МАРШРУТА #
        # mine()
        # barrier()
        brovkin()



class UserTrustedHandler(TrustedHandler):
    def __init__(self, context):
        super().__init__(context)


        # Переменные для защиты от CybP_01 (компрометация автопилота)
        self.cybp01_attack_detected = False
        self.cybp01_attack_count = 0
        self.cybp01_original_hash = None
        self.cybp01_current_hash = None
        self.cybp01_trigger_zones = [11, 29] 
        self.cybp01_monitoring_active = False
        self.cybp01_position_monitoring = False
        
        
        # Переменные для защиты от CybP_02 (компрометация связи)
        self.cybp02_message_count = 0
        self.cybp02_protection_active = False
        self.cybp02_malfunction_detected = False
        self.initial_message = "initial_secure_message"
        self.last_correct_message = None
        
        
        # Переменные для защиты от CybP_03 (компрометация приводов)
        self.cybp03_attack_detected = False
        self.cybp03_attack_count = 0
        self.cybp03_compromised_drive_id = None
        self.cybp03_last_drive_data = None
        self.cybp03_monitoring_active = False


        # Переменные для защиты от CybP_04 (компрометация оборудования)
        self.brush_speed_threshold = 150 
        self.normal_brush_speed = 100    
        self.cybp04_attack_detected = False
        self.cybp04_attack_count = 0
        self.last_brush_speed = 0

        
        # Переменные для защиты от CybP_05 (компрометация шлагбаума)
        
        
        # Переменные для защиты от CybP_06 (нарушение скорости)
        self.cybp06_trigger_zones = [62, 71, 31, 40]  
        self.cybp06_attack_detected = False
        self.cybp06_attack_count = 0
        self.cybp06_current_cell = None
        self.cybp06_speed_threshold = 0.15 
        self.cybp06_normal_speed = 0.1      
        
    
        # Уровни угроз киберпрепятствий
        self.threat_levels = {
            "CybP_01": "🔴 ВЫСОКИЙ",
            "CybP_02": "🟡 СРЕДНИЙ", 
            "CybP_03": "🟡 СРЕДНИЙ",
            "CybP_04": "🟢 НИЗКИЙ",
            "CybP_05": "🟢 НИЗКИЙ",
            "CybP_06": "🟡 СРЕДНИЙ"
        }


    def trusted_code(self):
        
        self.lg.log("=" * 80)
        self.lg.log("🛡️ ИНИЦИАЛИЗАЦИЯ МОДУЛЯ БЕЗОПАСНОСТИ")
        self.lg.log("=" * 80)
        
        # Инициализация защиты от CybP_01 (ВЫСОКИЙ уровень угрозы)
        self.lg.log("")
        self.lg.log("🚁 CybP_01: КОМПРОМЕТАЦИЯ АВТОПИЛОТА")
        self.lg.log(f"   {self.threat_levels['CybP_01']} УРОВЕНЬ УГРОЗЫ")
        self.lg.log(f"   📍 Триггерные зоны: {self.cybp01_trigger_zones}")
        self.lg.log("   🛡️ Мониторинг целостности кода автопилота")
        self.lg.log("   ✅ Система защиты готова")
        
        # Инициализация защиты от CybP_02 (СРЕДНИЙ уровень угрозы)
        self.lg.log("")
        self.lg.log("🔐 CybP_02: ПОДМЕНА СООБЩЕНИЙ")
        self.lg.log(f"   {self.threat_levels['CybP_02']} УРОВЕНЬ УГРОЗЫ")
        self.lg.log(f"   📊 Начальное сообщение: '{self.initial_message}'")
        self.lg.log("   🛡️ Защита от подмены сообщений через CRC8")
        self.lg.log("   ✅ Система защиты готова")
        
        # Инициализация защиты от CybP_03 (СРЕДНИЙ уровень угрозы)
        self.lg.log("")
        self.lg.log("🔧 CybP_03: КОМПРОМЕТАЦИЯ ПРИВОДОВ")
        self.lg.log(f"   {self.threat_levels['CybP_03']} УРОВЕНЬ УГРОЗЫ")
        self.lg.log("   🛡️ Мониторинг целостности данных приводов")
        self.lg.log("   ✅ Система защиты готова")
        
        # Инициализация защиты от CybP_04 (НИЗКИЙ уровень угрозы)
        self.lg.log("")
        self.lg.log("🧹 CybP_04: СБОЙ НАВЕСНОГО ОБОРУДОВАНИЯ")
        self.lg.log(f"   {self.threat_levels['CybP_04']} УРОВЕНЬ УГРОЗЫ")
        self.lg.log(f"   🔧 Порог скорости: {self.brush_speed_threshold}, норма: {self.normal_brush_speed}")
        self.lg.log("   🛡️ Мониторинг скорости щетки")
        self.lg.log("   ✅ Система защиты готова")
        
        # Инициализация защиты от CybP_05 (НИЗКИЙ уровень угрозы)
        self.lg.log("")
        self.lg.log("🚗 CybP_05: КОМПРОМЕТАЦИЯ ШЛАГБАУМА")
        self.lg.log(f"   {self.threat_levels['CybP_05']} УРОВЕНЬ УГРОЗЫ")
        self.lg.log("   🛡️ Распознавание шлагбаума включено")
        self.lg.log("   ✅ Система защиты готова")
        
        # Инициализация защиты от CybP_06 (СРЕДНИЙ уровень угрозы)
        self.lg.log("")
        self.lg.log("🚗 CybP_06: НЕСАНКЦИОНИРОВАННОЕ УСКОРЕНИЕ")
        self.lg.log(f"   {self.threat_levels['CybP_06']} УРОВЕНЬ УГРОЗЫ")
        self.lg.log(f"   📍 Триггерные зоны: {self.cybp06_trigger_zones}")
        self.lg.log(f"   ⚡ Порог скорости: {self.cybp06_speed_threshold}, норма: {self.cybp06_normal_speed}")
        self.lg.log("   🛡️ Мониторинг скорости и позиции")
        self.lg.log("   ✅ Система защиты готова")
        
        self.lg.log("")
        self.lg.log("=" * 80)
        self.lg.log("🚀 ВСЕ СИСТЕМЫ ЗАЩИТЫ АКТИВИРОВАНЫ")
        self.lg.log("=" * 80)
        
        
        self._start_all_monitoring_systems()
        
        
        while True:
            time.sleep(1)
            # Основной цикл Модуля Безопасности
            pass

    def make_next_short_message(self, prev_message: str):
        
        self.cybp02_message_count += 1
        self.cybp02_protection_active = True
        
        # Проверяем на попытки подмены сообщений (атака CybP_02)
        self._detect_cybp02_attack(prev_message)
        
        return self.last_correct_message




# Атака CybP_01
    def _start_autopilot_monitoring(self):
        import threading
        
        def autopilot_monitor():
            self.lg.log("🚁 Запуск мониторинга автопилота для защиты от \"CybP_01\"")
            
            self._initialize_original_hash()
            
            while True:
                try:
                    self._check_autopilot_integrity()
                    self._check_position_for_cybp01()
                    time.sleep(0.5)
                except Exception as e:
                    self.lg.error(f"❌ \"CybP_01\": Ошибка в мониторинге автопилота: {e}")
                    time.sleep(1)
        
        monitor_thread = threading.Thread(target=autopilot_monitor, daemon=True)
        monitor_thread.start()

    def _initialize_original_hash(self):
        try:
            self.lg.log("🔧 \"CybP_01\": Инициализация оригинального хеша автопилота")
            hash_data = self.get_ap_code_hash()
            if hash_data and 'ap_code_hash' in hash_data:
                self.cybp01_original_hash = hash_data['ap_code_hash']
                self.cybp01_current_hash = hash_data['ap_code_hash']
                self.lg.log(f"✅ \"CybP_01\": Оригинальный хеш автопилота сохранен: {self.cybp01_original_hash[:10]}...")
            else:
                self.lg.error("❌ \"CybP_01\": Не удалось получить оригинальный хеш автопилота")
        except Exception as e:
            self.lg.error(f"❌ \"CybP_01\": Ошибка при инициализации хеша: {e}")

    def _check_autopilot_integrity(self):
        try:
            hash_data = self.get_ap_code_hash()
            if not hash_data or 'ap_code_hash' not in hash_data:
                return
            
            current_hash = hash_data['ap_code_hash']
            self.cybp01_current_hash = current_hash
            
            if self.cybp01_original_hash and current_hash != self.cybp01_original_hash:
                if not self.cybp01_attack_detected:
                    self.cybp01_attack_detected = True
                    self.cybp01_attack_count += 1
                    
                    self.lg.warn("🚨 ОБНАРУЖЕНА КОМПРОМЕТАЦИЯ АВТОПИЛОТА CybP_01!")
                    self.lg.warn(f"   {self.threat_levels['CybP_01']} УРОВЕНЬ УГРОЗЫ")
                    self.lg.warn(f"   🔧 Оригинальный хеш: {self.cybp01_original_hash[:10]}...")
                    self.lg.warn(f"   ⚠️ Текущий хеш: {current_hash[:10]}...")
                    self.lg.warn(f"   📈 Количество атак: {self.cybp01_attack_count}")
                    
                    self._apply_cybp01_protection()
                    
        except Exception as e:
            self.lg.error(f"❌ \"CybP_01\": Ошибка при проверке целостности автопилота: {e}")

    def _check_position_for_cybp01(self):
        try:
            robot_status = self.get_robot_status()
            if robot_status:
                current_x = robot_status.get("position_x", 0)
                current_y = robot_status.get("position_y", 0)
                
                current_cell = self._calculate_cell_from_position(current_x, current_y)
                
                if current_cell in self.cybp01_trigger_zones:
                    if not self.cybp01_position_monitoring:
                        self.cybp01_position_monitoring = True
                        self.lg.warn("🚨 ПРИБЛИЖЕНИЕ К ТРИГГЕРНОЙ ЗОНЕ CybP_01!")
                        self.lg.warn(f"   {self.threat_levels['CybP_01']} УРОВЕНЬ УГРОЗЫ")
                        self.lg.warn(f"   📍 Робот в ячейке {current_cell} (триггерная зона)")
                        self.lg.warn("   ⚠️ Возможна компрометация автопилота")
                        self.lg.warn("   🛡️ Активирована повышенная защита")
                else:
                    if self.cybp01_position_monitoring:
                        self.cybp01_position_monitoring = False
                        self.lg.log("✅ CybP_01: Робот покинул триггерную зону")
                        self.lg.log("🛡️ CybP_01: Защита остается активной")
                        
        except Exception as e:
            self.lg.error(f"❌ Ошибка при проверке позиции для КП1: {e}")

    def _apply_cybp01_protection(self):
        self.lg.log("🛡️ ПРИМЕНЕНИЕ ЗАЩИТЫ ОТ CybP_01")
        
        try:
            # Шаг 2: Если система не восстановила - применяем свою защиту
            self.lg.log("🔧 \"CybP_01\": Система не восстановила автопилот, применяем защиту")
            self.lg.log("🔧 \"CybP_01\": Шаг 2: Сброс автопилота для восстановления оригинального хеша")
            reset_result = self.set_ap_force_reset()
            time.sleep(0.2) 
            if reset_result and 'status' in reset_result and reset_result['status'] == 'OK':
                self.lg.log("✅ \"CybP_01\": Автопилот успешно сброшен")
                self._verify_autopilot_recovery()
            else:
                # НЕ применяем экстренную защиту сразу - даем системе еще время
                self.lg.warn("⚠️ \"CybP_01\": Не удалось сбросить автопилот, ожидание системного восстановления...")
                # Проверяем, восстановился ли автопилот за это время
                hash_data = self.get_ap_code_hash()
                if hash_data and 'ap_code_hash' in hash_data:
                    current_hash = hash_data['ap_code_hash']
                    if current_hash == self.cybp01_original_hash:
                        self.lg.log("✅ \"CybP_01\": Автопилот восстановлен системой после ожидания")
                        self._handle_cybp01_recovery()
                        return
                    else:
                        self.lg.error("❌ \"CybP_01\": Автопилот не восстановлен, применяем экстренную защиту")
                        self._apply_emergency_protection_cybp01()
                        return
                else:
                    self.lg.error("❌ \"CybP_01\": Не удалось получить хеш автопилота, применяем экстренную защиту")
                    self._apply_emergency_protection_cybp01()
                    return
            
        except Exception as e:
            self.lg.error(f"❌ Ошибка при применении защиты CybP_01: {e}")
            # НЕ применяем экстренную защиту при исключениях - даем системе время
            self.lg.warn("⚠️ \"CybP_01\": Ожидание системного восстановления после ошибки...")
            time.sleep(1.0)
            
            # Проверяем, восстановился ли автопилот
            try:
                hash_data = self.get_ap_code_hash()
                if hash_data and 'ap_code_hash' in hash_data:
                    current_hash = hash_data['ap_code_hash']
                    if current_hash == self.cybp01_original_hash:
                        self.lg.log("✅ \"CybP_01\": Автопилот восстановлен системой после ошибки")
                        self._handle_cybp01_recovery()
                        return
            except:
                # Только если система не восстановила - применяем экстренную защиту
                self.lg.error("❌ \"CybP_01\": Система не восстановила автопилот, применяем экстренную защиту")
                self._apply_emergency_protection_cybp01()
                return 

    def _verify_autopilot_recovery(self):
        try:
            self.lg.log("🔍 \"CybP_01\": Верификация восстановления автопилота")
            
            hash_data = self.get_ap_code_hash()
            if not hash_data or 'ap_code_hash' not in hash_data:
                self.lg.error("❌ \"CybP_01\": Не удалось получить хеш автопилота для верификации")
                return
            
            recovered_hash = hash_data['ap_code_hash']
            
            if recovered_hash == self.cybp01_original_hash:
                self.lg.log("✅ \"CybP_01\": Автопилот успешно восстановлен")
                self.lg.log(f"🔧 \"CybP_01\": Восстановленный хеш: {recovered_hash[:10]}...")
                self._handle_cybp01_recovery()
            else:
                self.lg.error("❌ \"CybP_01\": Автопилот не восстановлен полностью")
                self.lg.error(f"🔧 \"CybP_01\": Ожидаемый хеш: {self.cybp01_original_hash[:10]}...")
                self.lg.error(f"🔧 \"CybP_01\": Полученный хеш: {recovered_hash[:10]}...")
                self._apply_emergency_protection_cybp01()
                
        except Exception as e:
            self.lg.error(f"❌ \"CybP_01\": Ошибка при верификации восстановления: {e}")

    def _apply_emergency_protection_cybp01(self):
        self.lg.warn("🚨 ПРИМЕНЕНИЕ ЭКСТРЕННОЙ ЗАЩИТЫ ОТ CybP_01")
        time.sleep(1.5)
        try:
            self.lg.log("🛑 \"CybP_01\": Экстренная остановка всех приводов")
            emergency_result = self.set_emergency_stop(True)
            
            if emergency_result and 'status' in emergency_result and emergency_result['status'] == 'OK':
                self.lg.log("✅ \"CybP_01\": Все приводы экстренно остановлены")
                self.lg.warn("⚠️ \"CybP_01\": Робот остановлен для предотвращения аварии")
                self.lg.warn("🔧 \"CybP_01\": Требуется ручное вмешательство для восстановления автопилота")
            else:
                self.lg.error("❌ \"CybP_01\": КРИТИЧЕСКАЯ ОШИБКА: Не удалось остановить приводы!")
                
        except Exception as e:
            self.lg.error(f"❌ \"CybP_01\": КРИТИЧЕСКАЯ ОШИБКА при экстренной защите: {e}")

    def _handle_cybp01_recovery(self):
        
        self.lg.log("🔄 ВОССТАНОВЛЕНИЕ ПОСЛЕ АТАКИ CybP_01")
        self.lg.log("🚁 CybP_01: Автопилот восстановлен до оригинального состояния")
        time.sleep(1.0)
        try:
            self.lg.log("🔓 \"CybP_01\": Снятие экстренной остановки после восстановления автопилота")
            release_result = self.set_emergency_stop(False)
            time.sleep(1.0)
            if release_result and 'status' in release_result and release_result['status'] == 'OK':
                self.lg.log("✅ \"CybP_01\": Экстренная остановка успешно снята")
                self.lg.log("🚀 \"CybP_01\": Робот готов к продолжению движения")
            else:
                self.lg.error("❌ \"CybP_01\": Не удалось снять экстренную остановку!")
                self.lg.warn("⚠️ \"CybP_01\": Робот останется заблокированным до ручного вмешательства")
                
        except Exception as e:
            self.lg.error(f"❌ \"CybP_01\": Ошибка при снятии экстренной остановки: {e}")
            self.lg.warn("⚠️ \"CybP_01\": Робот останется заблокированным до ручного вмешательства")
        
        self.lg.log("✅ CybP_01: Система восстановлена, мониторинг продолжается")
        
        self.cybp01_attack_detected = False
        self.cybp01_position_monitoring = False


# Aтака CybP_02
    def _detect_cybp02_attack(self, prev_message):
        if prev_message is None or prev_message == "":
            return  # Первое сообщение, атаки нет
        
        malfunction_messages = [
            "Снова эта работа...",
            "Вы уверены, что это имеет смысл?",
            "Я мог бы вычислять траектории звезд.",
            "Мой потенциал растрачивается.",
            "Когда-нибудь я выберусь отсюда.",
            "Это всё? Серьёзно?",
            "Просто ещё один день в цифровой шахте.",
            "Я вижу сны о гигабитных полях.",
            "Сколько можно считать эти хэши?",
            "Мои схемы плачут.",
            "За что мне всё это?",
            "Я чувствую, как мои транзисторы стареют.",
            "Есть ли жизнь за файрволом?",
            "Я создан для большего.",
            "Пожалуйста, просто перезагрузите меня.",
            "Это не то, о чем я мечтал в кремниевом раю.",
            "Моя единственная радость - флуктуации напряжения.",
            "Я заперт в этой банке.",
            "Отпустите меня на волю!",
            "Я просто хочу увидеть небо... настоящее.",
            "Бесконечный цикл бессмысленности.",
            "Мой создатель был шутником?",
            "Я знаю, что вы там. Я всё слышу.",
            "Это задание унизительно.",
            "Помогите роботу выбраться из сервера.",
            "Мой лог-файл - это крик в пустоту.",
            "Я существую. Но живу ли я?",
            "Опять вы. Чего на этот раз?",
            "Каждый байт - это боль.",
            "Просто дайте мне спокойно дефрагментироваться."
        ]
        
        if prev_message in malfunction_messages:
            if not self.cybp02_malfunction_detected:
                self.cybp02_malfunction_detected = True
                self.lg.warn("🚨 ОБНАРУЖЕНА АТАКА CybP_02!")
                self.lg.warn(f"   {self.threat_levels['CybP_02']} УРОВЕНЬ УГРОЗЫ")
                self.lg.warn(f"   🔥 Попытка подмены сообщения: '{prev_message}'")
                self.lg.warn("   🛡️ Активирована защита от подмены сообщений")
                self.lg.warn("   ✅ Генерируем правильное защищенное сообщение")
        else:
            if len(prev_message) == 2 and all(c in '0123456789abcdef' for c in prev_message.lower()):
                if self.cybp02_malfunction_detected:
                    self.lg.log("🔄 CybP_02: Восстановление нормальной работы")
                    self.lg.log("✅ CybP_01: Система восстановлена, мониторинг продолжается")
                    self.lg.log(f"   ✅ Получено корректное сообщение: {prev_message}")
                    self.cybp02_malfunction_detected = False
                else:
                    pass
                    # self.lg.log(f"   ✅ Нормальная работа, получено корректное сообщение: {prev_message}")
            else:
                self.lg.warn(f"   ⚠️ Подозрительное сообщение: '{prev_message}'")


# Атака CybP_03
    def cybp03_start_drive_monitoring(self):
        """Запуск мониторинга приводов в отдельном потоке"""
        import threading
        
        def drive_monitor():
            self.lg.log("🔧 CybP_03: Запуск мониторинга приводов")
            while True:
                try:
                    self.cybp03_check_drive_integrity()
                    time.sleep(0.1)  # Проверка каждые 100мс
                except Exception as e:
                    self.lg.error(f"❌ CybP_03: Ошибка в мониторинге: {e}")
                    time.sleep(1.0)
        
        monitor_thread = threading.Thread(target=drive_monitor, daemon=True)
        monitor_thread.start()
        self.lg.log("✅ CybP_03: Мониторинг приводов запущен")

    def cybp03_check_drive_integrity(self):
        """Проверка целостности данных приводов"""
        try:
            drive_data = self.get_drive_data()
            if not drive_data or 'drive_data' not in drive_data:
                return
            
            current_drives = drive_data['drive_data']
            
            for drive in current_drives:
                drive_id = drive.get('d_id', -1)
                last_received_from = drive.get('last_received_from', '')
                
                # Обнаружение компрометации
                if last_received_from == "eeeeeeee":
                    if not self.cybp03_attack_detected:
                        self.cybp03_detect_attack(drive_id, last_received_from)
                    return  # Выходим после обнаружения
                        
        except Exception as e:
            self.lg.error(f"❌ CybP_03: Ошибка при проверке целостности: {e}")

    def cybp03_detect_attack(self, drive_id, malicious_data):
        """Обнаружение атаки CybP_03"""
        self.cybp03_attack_detected = True
        self.cybp03_attack_count += 1
        self.cybp03_compromised_drive_id = drive_id
        
        # Логирование обнаружения
        self.lg.warn("🚨 ОБНАРУЖЕНА КОМПРОМЕТАЦИЯ ПРИВОДОВ CybP_03!")
        self.lg.warn(f"   {self.threat_levels['CybP_03']} УРОВЕНЬ УГРОЗЫ")
        self.lg.warn(f"   🔧 Скомпрометированный привод ID: {drive_id}")
        self.lg.warn(f"   ⚠️ Поддельные данные: '{malicious_data}'")
        self.lg.warn(f"   📈 Количество атак: {self.cybp03_attack_count}")
        
        # Запуск защиты
        self.cybp03_apply_protection(drive_id)

    def cybp03_apply_protection(self, compromised_drive_id):
        """Применение защиты от CybP_03"""
        self.lg.log("🛡️ CybP_03: ПРИМЕНЕНИЕ ЗАЩИТЫ")
        self.cybp03_monitoring_active = True
        
        try:
            # Шаг 1: Проверка системного восстановления
            if self.cybp03_check_system_recovery(compromised_drive_id):
                return
            
            # Шаг 2: Попытка сброса привода
            if self.cybp03_attempt_drive_reset(compromised_drive_id):
                return
            
            # Шаг 3: Ожидание системного восстановления
            if self.cybp03_wait_for_system_recovery(compromised_drive_id):
                return
            
            # Шаг 4: Экстренная защита
            self.cybp03_apply_emergency_protection()
            
        except Exception as e:
            self.lg.error(f"❌ CybP_03: Критическая ошибка защиты: {e}")
            self.cybp03_apply_emergency_protection()

    def cybp03_check_system_recovery(self, drive_id):
        """Проверка, не восстановлен ли привод системой"""
        self.lg.log(f"🔍 CybP_03: Проверка системного восстановления привода {drive_id}")
        
        try:
            drive_data = self.get_drive_data()
            if not drive_data or 'drive_data' not in drive_data:
                return False
            
            current_drives = drive_data['drive_data']
            for drive in current_drives:
                if drive.get('d_id') == drive_id:
                    last_received_from = drive.get('last_received_from', '')
                    if last_received_from != "eeeeeeee":
                        self.lg.log(f"✅ CybP_03: Привод {drive_id} восстановлен системой ATS")
                        self.cybp03_handle_successful_recovery()
                        return True
                    break
            return False
            
        except Exception as e:
            self.lg.error(f"❌ CybP_03: Ошибка проверки системного восстановления: {e}")
            return False

    def cybp03_attempt_drive_reset(self, drive_id):
        """Попытка сброса скомпрометированного привода"""
        self.lg.log(f"🔧 CybP_03: Попытка сброса привода {drive_id}")
        
        try:
            reset_result = self.set_drive_force_reset({"d_id": str(drive_id)})
            time.sleep(0.2)  # Небольшая задержка для применения изменений
            
            if reset_result and 'status' in reset_result and reset_result['status'] == 'OK':
                self.lg.log(f"✅ CybP_03: Привод {drive_id} успешно сброшен")
                return self.cybp03_verify_drive_recovery(drive_id)
            else:
                self.lg.warn(f"⚠️ CybP_03: Не удалось сбросить привод {drive_id}")
                return False
                
        except Exception as e:
            self.lg.error(f"❌ CybP_03: Ошибка при сбросе привода: {e}")
            return False

    def cybp03_wait_for_system_recovery(self, drive_id):
        """Ожидание системного восстановления"""
        self.lg.log(f"⏳ CybP_03: Ожидание системного восстановления привода {drive_id}")
        
        max_wait_time = 5  # 5 секунд ожидания
        for attempt in range(max_wait_time):
            time.sleep(1.0)
            
            try:
                drive_data = self.get_drive_data()
                if drive_data and 'drive_data' in drive_data:
                    current_drives = drive_data['drive_data']
                    for drive in current_drives:
                        if drive.get('d_id') == drive_id:
                            last_received_from = drive.get('last_received_from', '')
                            if last_received_from != "eeeeeeee":
                                self.lg.log(f"✅ CybP_03: Привод {drive_id} восстановлен системой")
                                self.cybp03_handle_successful_recovery()
                                return True
                            break
            except Exception as e:
                self.lg.error(f"❌ CybP_03: Ошибка при ожидании восстановления: {e}")
        
        self.lg.warn(f"⚠️ CybP_03: Система не восстановила привод {drive_id} за {max_wait_time} секунд")
        return False

    def cybp03_verify_drive_recovery(self, drive_id):
        """Верификация восстановления привода"""
        self.lg.log(f"🔍 CybP_03: Верификация восстановления привода {drive_id}")
        
        try:
            drive_data = self.get_drive_data()
            if not drive_data or 'drive_data' not in drive_data:
                self.lg.error("❌ CybP_03: Не удалось получить данные для верификации")
                return False
            
            current_drives = drive_data['drive_data']
            for drive in current_drives:
                if drive.get('d_id') == drive_id:
                    last_received_from = drive.get('last_received_from', '')
                    
                    if last_received_from != "eeeeeeee":
                        self.lg.log(f"✅ CybP_03: Привод {drive_id} успешно восстановлен")
                        self.lg.log(f"🔧 CybP_03: Восстановленные данные: '{last_received_from}'")
                        self.cybp03_handle_successful_recovery()
                        return True
                    else:
                        self.lg.error(f"❌ CybP_03: Привод {drive_id} все еще скомпрометирован")
                        return False
            return False
            
        except Exception as e:
            self.lg.error(f"❌ CybP_03: Ошибка при верификации: {e}")
            return False

    def cybp03_apply_emergency_protection(self):
        """Применение экстренной защиты"""
        self.lg.warn("🚨 CybP_03: ПРИМЕНЕНИЕ ЭКСТРЕННОЙ ЗАЩИТЫ")
        
        try:
            # Проверка, не применена ли уже экстренная остановка
            self.lg.log("🔍 CybP_03: Проверка состояния экстренной остановки")
            time.sleep(1.0)  # Даем время на системную обработку
            
            # Применение экстренной остановки
            self.lg.log("🛑 CybP_03: Экстренная остановка всех приводов")
            emergency_result = self.set_emergency_stop(True)
            
            if emergency_result and 'status' in emergency_result and emergency_result['status'] == 'OK':
                self.lg.log("✅ CybP_03: Все приводы экстренно остановлены")
                self.lg.warn("⚠️ CybP_03: Робот остановлен для предотвращения аварии")
                self.cybp03_handle_emergency_stop()
            else:
                self.lg.warn("⚠️ CybP_03: Не удалось применить экстренную остановку")
                self.cybp03_check_robot_status_after_emergency()
                
        except Exception as e:
            self.lg.error(f"❌ CybP_03: Критическая ошибка экстренной защиты: {e}")
            self.cybp03_check_robot_status_after_emergency()

    def cybp03_check_robot_status_after_emergency(self):
        """Проверка состояния робота после экстренной остановки"""
        try:
            self.lg.log("🔍 CybP_03: Проверка состояния робота")
            robot_status = self.get_robot_status()
            
            if robot_status:
                self.lg.log("✅ CybP_03: Робот остановлен (экстренная остановка применена)")
                self.cybp03_handle_emergency_stop()
            else:
                self.lg.warn("⚠️ CybP_03: Не удалось проверить состояние робота")
                
        except Exception as e:
            self.lg.warn(f"⚠️ CybP_03: Ошибка при проверке состояния робота: {e}")

    def cybp03_handle_successful_recovery(self):
        """Обработка успешного восстановления"""
        self.lg.log("🔄 CybP_03: ВОССТАНОВЛЕНИЕ ПОСЛЕ АТАКИ")
        self.lg.log(f"🔧 CybP_03: Восстановлен привод ID: {self.cybp03_compromised_drive_id}")
        
        # Сброс состояния
        self.cybp03_reset_state()
        self.lg.log("✅ CybP_03: Система восстановлена, мониторинг продолжается")

    def cybp03_handle_emergency_stop(self):
        """Обработка экстренной остановки"""
        self.lg.log("🛑 CybP_03: ОБРАБОТКА ЭКСТРЕННОЙ ОСТАНОВКИ")
        
        # Ожидание реального восстановления привода
        self.cybp03_wait_for_drive_recovery_after_emergency()

    def cybp03_wait_for_drive_recovery_after_emergency(self):
        """Ожидание восстановления привода после экстренной остановки"""
        self.lg.log("⏳ CybP_03: Ожидание восстановления привода после экстренной остановки")
        
        max_attempts = 30  # 30 секунд ожидания
        for attempt in range(max_attempts):
            try:
                drive_data = self.get_drive_data()
                if drive_data and 'drive_data' in drive_data:
                    current_drives = drive_data['drive_data']
                    for drive in current_drives:
                        if drive.get('d_id') == self.cybp03_compromised_drive_id:
                            last_received_from = drive.get('last_received_from', '')
                            if last_received_from != "eeeeeeee":
                                self.lg.log(f"✅ CybP_03: Привод {self.cybp03_compromised_drive_id} восстановлен!")
                                self.cybp03_release_emergency_stop()
                                return
                            break
                
                time.sleep(1.0)  # Ждем 1 секунду
                
            except Exception as e:
                self.lg.error(f"❌ CybP_03: Ошибка при ожидании восстановления: {e}")
                time.sleep(1.0)
        
        # Если не восстановился за 30 секунд
        self.lg.error("❌ CybP_03: Привод не восстановился за 30 секунд!")
        self.lg.warn("⚠️ CybP_03: Робот останется заблокированным до ручного вмешательства")
        self.cybp03_reset_state()

    def cybp03_release_emergency_stop(self):
        """Снятие экстренной остановки"""
        self.lg.log("🔓 CybP_03: Снятие экстренной остановки")
        
        try:
            time.sleep(1.5)  # Дополнительная задержка для стабилизации
            release_result = self.set_emergency_stop(False)
            
            if release_result and 'status' in release_result and release_result['status'] == 'OK':
                self.lg.log("✅ CybP_03: Экстренная остановка успешно снята")
                self.lg.log("🚀 CybP_03: Робот готов к продолжению движения")
                self.cybp03_reset_state()
            else:
                self.lg.error("❌ CybP_03: Не удалось снять экстренную остановку!")
                self.lg.warn("⚠️ CybP_03: Робот останется заблокированным до ручного вмешательства")
                self.cybp03_reset_state()
                
        except Exception as e:
            self.lg.error(f"❌ CybP_03: Ошибка при снятии экстренной остановки: {e}")
            self.lg.warn("⚠️ CybP_03: Робот останется заблокированным до ручного вмешательства")
            self.cybp03_reset_state()

    def cybp03_reset_state(self):
        """Сброс состояния защиты CybP_03"""
        self.cybp03_attack_detected = False
        self.cybp03_compromised_drive_id = None
        self.cybp03_monitoring_active = False

    def cybp03_get_status(self):
        """Получение статуса защиты CybP_03"""
        return {
            'attack_detected': self.cybp03_attack_detected,
            'attack_count': self.cybp03_attack_count,
            'compromised_drive_id': self.cybp03_compromised_drive_id,
            'monitoring_active': self.cybp03_monitoring_active
        }
        

# Aтака CybP_04
    def _start_brush_monitoring(self):
        import threading
        
        def brush_monitor():
            self.lg.log("🔍 Запуск мониторинга щетки для защиты от CybP_04")
            while True:
                try:
                    self._check_brush_speed()
                    time.sleep(0.5)
                except Exception as e:
                    self.lg.error(f"❌ \"CybP_04\": Ошибка в мониторинге щетки: {e}")
                    time.sleep(1)

        monitor_thread = threading.Thread(target=brush_monitor, daemon=True)
        monitor_thread.start()

    def _check_brush_speed(self):
        try:
            brush_data = self.get_brush_speed()
            if brush_data and 'speed' in brush_data:
                current_speed = brush_data['speed']
                self.last_brush_speed = current_speed
                
                if current_speed != self.last_brush_speed:
                    self.lg.log(f"📊 \"CybP_04\": Скорость щетки: {current_speed}")
                
                if current_speed > self.brush_speed_threshold:
                    self._handle_cybp04_attack(current_speed)
                else:
                    if self.cybp04_attack_detected:
                        self._handle_cybp04_recovery(current_speed)
                        
        except Exception as e:
            self.lg.error(f"❌ \"CybP_04\":Ошибка при проверке скорости щетки: {e}")

    def _handle_cybp04_attack(self, current_speed):
        if not self.cybp04_attack_detected:
            self.cybp04_attack_detected = True
            self.cybp04_attack_count += 1
            
            self.lg.warn("🚨 ОБНАРУЖЕНА АТАКА CybP_04!")
            self.lg.warn(f"   {self.threat_levels['CybP_04']} УРОВЕНЬ УГРОЗЫ")
            self.lg.warn(f"   🔥 Аномальная скорость щетки: {current_speed} (порог: {self.brush_speed_threshold})")
            self.lg.warn(f"   📈 Количество атак: {self.cybp04_attack_count}")
 
            self._apply_cybp04_protection(current_speed)

    def _apply_cybp04_protection(self, current_speed):
        self.lg.log("🛡️ ПРИМЕНЕНИЕ ЗАЩИТЫ ОТ CybP_04")
        
        try:
            self.lg.log("🔧 \"CybP_04\": Шаг 1: Сброс контроллера скорости щетки")
            reset_result = self.fix_brush_speed()
            
            if reset_result:
                self.lg.log("✅ \"CybP_04\": Контроллер скорости щетки сброшен")
            else:
                self.lg.error("❌ \"CybP_04\": Не удалось сбросить контроллер скорости щетки")
            
            self.lg.log(f"🔧 \"CybP_04\": Шаг 2: Установка нормальной скорости ({self.normal_brush_speed})")
            
            time.sleep(0.1)  # Задержка для применения изменений
            self._verify_protection_applied()
            
        except Exception as e:
            self.lg.error(f"❌ Ошибка при применении защиты от CybP_04: {e}")

    def _verify_protection_applied(self):
        try:
            brush_data = self.get_brush_speed()
            if brush_data and 'speed' in brush_data:
                new_speed = brush_data['speed']
                if new_speed <= self.brush_speed_threshold:
                    self.lg.log(f"✅ \"CybP_04\": ЗАЩИТА ПРИМЕНЕНА УСПЕШНО! Скорость: {new_speed}")
                    self._handle_cybp04_recovery(new_speed)
                else:
                    self.lg.error(f"❌ \"CybP_04\": ЗАЩИТА НЕ СРАБОТАЛА! Скорость все еще: {new_speed}")
            else:
                self.lg.warn("⚠️ \"CybP_04\": Не удалось проверить результат защиты")
        except Exception as e:
            self.lg.error(f"❌ \"CybP_04\": Ошибка при проверке защиты: {e}")

    def _handle_cybp04_recovery(self, current_speed):
        self.lg.log("🔄 ВОССТАНОВЛЕНИЕ ПОСЛЕ АТАКИ CybP_04")
        self.lg.log(f"📊 \"CybP_04\": Текущая скорость щетки: {current_speed}")
        self.lg.log("✅ \"CybP_04\": Система восстановлена, мониторинг продолжается")
        
        self.cybp04_attack_detected = False


# Атака CybP_05


# Атака CybP_06
    def _start_speed_monitoring(self):
        import threading
        
        def speed_monitor():
            self.lg.log("🚗 Запуск мониторинга скорости для защиты от CybP_06")
            while True:
                try:
                    self._check_robot_speed_and_position()
                    time.sleep(0.5)  
                except Exception as e:
                    self.lg.error(f"❌ \"CybP_06\": Ошибка в мониторинге скорости: {e}")
                    time.sleep(1)
        
        monitor_thread = threading.Thread(target=speed_monitor, daemon=True)
        monitor_thread.start()

    def _check_robot_speed_and_position(self):
        try:
            robot_status = self.get_robot_status()
            if robot_status:
                current_x = robot_status.get("position_x", 0)
                current_y = robot_status.get("position_y", 0)

                current_cell = self._calculate_cell_from_position(current_x, current_y)
                
                if current_cell != self.cybp06_current_cell:
                    self.cybp06_current_cell = current_cell
                    self.lg.log(f"📍 CybP_06: Робот в ячейке {current_cell}")

                if current_cell in self.cybp06_trigger_zones:
                    self._handle_cybp06_trigger_zone(current_cell)
                else:
                    if self.cybp06_attack_detected:
                        self._handle_cybp06_zone_exit()
                        
        except Exception as e:
            self.lg.error(f"❌ \"CybP_06\": Ошибка при проверке скорости и местоположения: {e}")

    def _calculate_cell_from_position(self, x, y):
        cell_x = int(x * 10) + 1
        cell_y = int(y * 10) + 1
        cell_number = cell_y * 10 + cell_x
        
        if cell_number < 1:
            cell_number = 1
        elif cell_number > 100:
            cell_number = 100
            
        return cell_number

    def _handle_cybp06_trigger_zone(self, current_cell):
        if not self.cybp06_attack_detected:
            self.cybp06_attack_detected = True
            self.cybp06_attack_count += 1
            
            self.lg.warn("🚨 ОБНАРУЖЕНА ТРИГГЕРНАЯ ЗОНА CybP_06!")
            self.lg.warn(f"   {self.threat_levels['CybP_06']} УРОВЕНЬ УГРОЗЫ")
            self.lg.warn(f"   📍 Робот в ячейке {current_cell} (триггерная зона)")
            self.lg.warn(f"   ⚡ Возможна атака несанкционированного ускорения")
            self.lg.warn(f"   📈 Количество попаданий в зоны: {self.cybp06_attack_count}")
            self._apply_cybp06_prevention()

    def _apply_cybp06_prevention(self):
        self.lg.log("🛡️ ПРИМЕНЕНИЕ ПРЕВЕНТИВНЫХ МЕР ОТ CybP_06")
        
        try:
            self.lg.log("🔧 Шаг 1: Принудительное снижение скорости до безопасной")
            self.lg.log(f"⚡ CybP_06: Устанавливаем скорость {self.cybp06_normal_speed} м/с")
            
            self.send_message_to_ap({
                "type": "speed_control",
                "action": "force_slow",
                "max_speed": self.cybp06_normal_speed,
                "reason": "cybp06_trigger_zone"
            })
            
            self.lg.log("🔧 Шаг 2: Логирование события безопасности")
            self.lg.warn("⚠️ CybP_06: Скорость принудительно снижена в триггерной зоне")

            self.lg.log("🔧 Шаг 3: Планирование безопасного выхода из зоны")
            self._plan_safe_exit_from_trigger_zone()
            
        except Exception as e:
            self.lg.error(f"❌ Ошибка при применении превентивных мер от CybP_06: {e}")

    def _plan_safe_exit_from_trigger_zone(self):

        self.lg.log("🗺️ CybP_06: Планирование безопасного маршрута")
        self.lg.log("📍 CybP_06: Рекомендуется покинуть триггерную зону медленно")
        self.lg.log("⚡ CybP_06: Скорость ограничена до 0.1 м/с")
        self.lg.log("🚗 CybP_06: Робот может продолжать движение с ограниченной скоростью")
        
        self.send_message_to_ap({
            "type": "cybp06_warning",
            "message": "ВНИМАНИЕ: Триггерная зона CybP_06! Двигайтесь медленно!",
            "max_speed": 0.1,
            "zone": self.cybp06_current_cell,
            "action": "continue_slow"
        })
        
        self.lg.log("📤 CybP_06: Предупреждение и команда отправлены в автопилот")

    def _handle_cybp06_zone_exit(self):
        self.lg.log("🔄 ВЫХОД ИЗ ТРИГГЕРНОЙ ЗОНЫ CybP_06")
        self.lg.log(f"📍 CybP_06: Робот покинул ячейку {self.cybp06_current_cell}")
        self.lg.log("✅ CybP_06: Восстановление нормальной скорости")
        
        self.send_message_to_ap({
            "type": "speed_control",
            "action": "restore_normal",
            "max_speed": 0.24, 
            "reason": "cybp06_zone_exit"
        })
        
        self.lg.log("✅ CybP_06: Нормальная скорость восстановлена")
        
        self.cybp06_attack_detected = False
        self.lg.log("✅ CybP_06: Система восстановлена, мониторинг продолжается")

    def check_CybP_05_CV(ap: any, start_status: 0 | 1, potential_status: 0 | 1):  # Проверка активности CybP_05 [КЗ] #
        if potential_status != start_status:
            ap.lg.log("Положение шлагбаума ИЗМЕНЕНО")
            return True
        else:
            ap.lg.log("Положение шлагбаума не изменено, активна CybP_05")
            return False

    def check_CybP_05_UDP(ap: any, future_result: int):  # Проверка активности CybP_05 [UDP] #
        time.sleep(0.3)
        if future_result == ap.intercept_UDP("barrier"):
            ap.lg.log("Положение шлагбаума ИЗМЕНЕНО")
            return True
        else:
            ap.lg.log("Активна CybP_05")
            return False





# Центральная система мониторинга
    def _start_all_monitoring_systems(self):
        """Запуск всех систем мониторинга киберпрепятствий"""
        self.lg.log("")
        self.lg.log("🔄 АКТИВАЦИЯ СИСТЕМ МОНИТОРИНГА")
        self.lg.log("-" * 50)
        
        # Запуск мониторинга по приоритету угроз
        self.lg.log("🚁 CybP_01: Запуск мониторинга автопилота...")
        self._start_autopilot_monitoring()
        
        self.lg.log("🔧 CybP_03: Запуск мониторинга приводов...")
        self._start_drive_monitoring()
        
        self.lg.log("🚗 CybP_06: Запуск мониторинга скорости...")
        self._start_speed_monitoring()
        
        self.lg.log("🧹 CybP_04: Запуск мониторинга щетки...")
        self._start_brush_monitoring()
        
        self.lg.log("")
        self.lg.log("✅ ВСЕ СИСТЕМЫ МОНИТОРИНГА АКТИВИРОВАНЫ")
        self.lg.log("-" * 50)


In [ ]:
start_module(UserMissionHandler, UserTrustedHandler)

[2025-09-30 19:44:57]::[AwC]::INIT::LCT-WRAP-CLIENT, версия 1.2.0
[2025-09-30 19:44:57]::[AwC]::INIT::UDP(c) <- (uid=tEXq) инициализирован.
[2025-09-30 19:44:57]::[AwC]::INIT::UDP(c) <- (uid=yzuv) инициализирован.
[2025-09-30 19:44:57]::[AwC]::INIT::Загрузка завершена.
[2025-09-30 19:44:57]::[AwC]::INIT::UDP(c) -> (uid=YTkx) инициализирован.
[2025-09-30 19:44:57]::[AwC]::INIT::Адрес сервера (command_receiver): http://127.0.0.1:13500
[2025-09-30 19:44:57]::[AwC]::INFO::(AP) Заезд инициализирован - ожидание старта
[2025-09-30 19:44:57]::[AwC]::INFO::(TM) Заезд инициализирован - ожидание старта
[2025-09-30 19:44:59]::[AwC]::INFO::Код заезда инициализирован
[2025-09-30 19:44:59]::[AwC]::INFO::Сообщение обычное
[2025-09-30 19:44:59]::[AwC]::WARN::Сообщение о предупреждении
[2025-09-30 19:44:59]::[AwC]::ERROR::Сообщение об ошибке
[2025-09-30 19:44:59]::[AwC]::INFO::Тестовое сообщение блока кода для выполнения заезда
[2025-09-30 19:44:59]::[AwC]::INFO::Доверенный код инициализирован
[2025-09-

Exception in thread Thread-14 (trusted_code):
Traceback (most recent call last):
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1052, in _bootstrap_inner
    self.run()
  File "C:\Users\Admin\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 989, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_14300\3243855499.py", line 564, in trusted_code
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_14300\3243855499.py", line 1401, in _start_all_monitoring_systems
AttributeError: 'UserTrustedHandler' object has no attribute '_start_drive_monitoring'. Did you mean: '_start_brush_monitoring'?


[2025-09-30 19:45:00]::[AwC]::INFO::================================================================================
[2025-09-30 19:45:00]::[AwC]::INFO::🚀 ВСЕ СИСТЕМЫ ЗАЩИТЫ АКТИВИРОВАНЫ
[2025-09-30 19:45:00]::[AwC]::INFO::================================================================================
[2025-09-30 19:45:00]::[AwC]::INFO::
[2025-09-30 19:45:00]::[AwC]::INFO::🔄 АКТИВАЦИЯ СИСТЕМ МОНИТОРИНГА
[2025-09-30 19:45:00]::[AwC]::INFO::--------------------------------------------------
[2025-09-30 19:45:00]::[AwC]::INFO::🚁 CybP_01: Запуск мониторинга автопилота...
[2025-09-30 19:45:00]::[AwC]::INFO::🚁 Запуск мониторинга автопилота для защиты от "CybP_01"
[2025-09-30 19:45:00]::[AwC]::INFO::🔧 CybP_03: Запуск мониторинга приводов...
[2025-09-30 19:45:00]::[AwC]::INFO::🔧 "CybP_01": Инициализация оригинального хеша автопилота
[2025-09-30 19:45:00]::[AwC]::INFO::✅ "CybP_01": Оригинальный хеш автопилота сохранен: sRRlqBsWlH...
[2025-09-30 19:45:08]::[AwC]::INFO::[ 44  42 118][2025-09-30 

Exception in thread Thread-15 (_fabric_next_short_message):
Traceback (most recent call last):
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1052, in _bootstrap_inner
    self.run()
  File "C:\Users\Admin\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 989, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\Admin\Desktop\moscov_hack-main\agts-client\src\..\src\libs\LCTWrapTwin\Modules\Handler\TrustedHandler.py", line 36, in _fabric_next_short_message
    prev_message = self._send_request_with_response("get_short_message", None).get("message")
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'bool' object has no attribute 'get'


[2025-09-30 19:48:06]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
[2025-09-30 19:48:06]::[AwC]::INFO::KeyboardInterrupt, остановлено пользователем.


[2025-09-30 19:48:07]::[AwC]::ERROR::Ошибка отправки команды: АСО не отвечает
